In [1]:
import os
os.chdir("..")

from funchub.math import custom_round
import re

def parse_answer(answer, pattern:str="####"):
    if pattern=="####":
        answer = answer.split("####")[-1]
        answer = answer.strip().strip("\n").strip('\\n')
        # 32,333 -> 32333
        answer = answer.replace(",", "")

        # get the last number
        try:
            answer = re.findall(r"[-+]?\d*\.\d+|\d+", answer)[-1]
        except:
            answer = 0
    elif pattern=="answer is":
        answer = answer.split("answer is")[-1]
        answer = answer.strip().strip("\n").strip('\\n')

        # 32,333 -> 32333
        answer = answer.replace(",", "")

        # get the last number
        try:
            answer = re.findall(r"[-+]?\d*\.\d+|\d+", answer)[-1]
        except:
            answer = 0

    return answer

def accuracy(pred, true, type = "exact"):
    if len(pred) < len(true):
        true = true[:len(pred)]

    correct = 0
    for p, t in zip(pred, true):
        try:
            if type == "exact":
                if float(p) == float(t):
                    correct += 1
            elif type == "round":
                if round(float(p), 2) == custom_round(float(t), 2):
                    correct += 1
            elif type == "approx":
                # 1% error tolerance, e.g. 1000 -> 990 ~ 1010
                if abs(float(p) - float(t)) <= abs(float(t)) * 0.001:
                    correct += 1
        except ValueError:
            pass
    
    return correct / len(pred)

In [8]:
import json

target_path = "data/funcqa/funcqa_mh.json"
eval_path = "outputs/funcqa_mh/inference-7B-None-baseline-funcqa_mh-0.jsonl"


with open(target_path, "r") as f:
    data = json.load(f)

answer = [d["answer"] for d in data]

with open(eval_path, "r") as f:
    data = [json.loads(line) for line in f]

pred = [parse_answer(d["generation"], pattern="####") for d in data]

print(pred)
print(answer[:len(pred)])

print("Accuracy: ", accuracy(pred, answer[:len(pred)], type="approx"))

['19.6', '24.96', '45.88', '100000000']
['32.38', '43.5694625', '92.256', '1556750.9']
Accuracy:  0.0


In [9]:
dummmy_answer = ['32.3799', '43', '100', '1556750']

print("Accuracy: ", accuracy(dummmy_answer, answer[:len(dummmy_answer)], type="approx"))

Accuracy:  0.5
